# 12 - Strojno učenje za kategorizacijo podatkov
<p style="font-weight: 600; text-align: center;">
Matematično-fizikalni praktikum, avgust 2024 <br>
Luka Skeledžija, 28201079
</p>

<style>
.MJXc-display{
    display: inline-block !important;
    width: 100%;
}
@media print {
    .pagebreak { page-break-before: always; } /* page-break-after works, as well */
}

img{
    width: 100%;
    max-width: 600px !important;
    margin: auto !important;
}

body {
    overflow: hidden;
    max-width: 600px;
    margin: auto;
}

::-webkit-scrollbar {
  width: 0px;
}

table{
    width: 100%;
}

h1 {
    text-transform: uppercase;
    text-align: center;
    background: #222222;
    color: white;
    padding: 8px;
}

blockquote {
    margin-left: 0em!important;
    margin-right: 0em!important;

}

.jp-RenderedHTMLCommon pre, .jp-RenderedHTMLCommon code {

    background-color: var(--jp-layout-color2)!important;
}

.jp-RenderedHTMLCommon pre{
    margin: 0.5em 0em!important;
    padding: 0em 1.5em!important;
}

body {
    counter-reset: h2counter;
}
h1 {
    counter-reset: h2counter;
}
h2:before {
    content: counter(h2counter) ".\0000a0\0000a0";
    counter-increment: h2counter;
    counter-reset: h3counter;
}
h3:before {
    counter-increment: h3counter;
    content: counter(h2counter) "." counter(h3counter) ".\0000a0\0000a0";
  
}




</style>


---

## Uvod

Dandanes je uporaba različnih algoritmov strojnega učenja (Machine Learning, ML) v 
znanosti že rutinsko opravilo. Poznamo tri osnovne vrste stojnega učenja:

  - Nadzorovano učenje (Supervised learning):
    - Klasifikacija (Classification): sortiranje v različne kategorije.
    - Regresija (Regression): modeliranje oz. `fitanje' napovedi.
  - Nenadzorovano učenje ( npr. sam najdi kategorije).
  - Stimulirano učenje ( Artificial Intelligence v ožjem pomenu besede).

V fiziki (in tej nalogi), se tipično ukvarjamo s prvo kategorijo, bodisi za identifikacijo novih pojavov/delcev/\ldots ali pa za ekstrakcijo napovedi (netrivialnih funkcijskih odvisnosti etc).

ML algoritmi imajo prednost pred klasičnim pristopom, da lahko učinkovito razdrobijo kompleksen problem na enostavne elemente in ga ustrezno opišejo:

  - pomisli na primer, kako bi bilo težko kar predpostaviti/uganiti 
  pravo analitično funkcijo v več dimenzijah ( in je npr. uporaba zlepkov 
  (spline interpolacija) mnogo lažja in boljša ).
  - Pri izbiri/filtriranju velike količine podatkov z mnogo lastnostmi 
  (npr dogodki pri trkih na LHC) je zelo težko najti količine, ki optimalno 
  ločijo signal od ozadnja, upoštevati vse korelacije in najti optimalno kombinacijo 
  le-teh

Če dodamo malce matematičnega formalizma strojnega učenja: Predpostavi, da imamo na voljo nabor primerov 
$\mathcal{D}=\left\{\left(\mathbf{x}_{k}, y_{k}\right)\right\}_{k=1 . N},$ kjer je
$\mathbf{x}_{k}=\left(x_{k}^{1}, \ldots, x_{k}^{M}\right)$ naključno izbrani vektor
 $M$ lastnosti (karakteristik) in je $\mathbf{y}_{k} = \left(y_{k}^{1}, \ldots, y_{k}^{Q}\right)$
 vektor $Q$ ciljnih vrednosti, ki so lahko bodisi binarne ali pa realna 
 števila. 
 Vrednosti $\left(\mathbf{x}_{k}, \mathbf{y}_{k}\right)$ so neodvisne in porazdeljene po neki
 neznani porazdelitvi $P(\cdot, \cdot) .$ Cilj ML metode je določiti (priučiti) funkcijo
 $h: \mathbb{R}^{Q} \rightarrow \mathbb{R}$, ki minimizira pričakovano vrednost **funkcije
 izgube (expected loss)** $$\mathcal{L}(h)=\mathbb{E}\, L(\mathbf{y}, \mathbf{h}(\mathbf{x})) = \frac{1}{N}\sum\limits_{k=1}^{N} L(\mathbf{y_k}, \mathbf{h}(\mathbf{x_k})). $$ 
 Tu je $L(\cdot, \cdot)$  gladka funkcija, ki opisuje oceno za kvaliteto napovedi, 
 pri čemer so  vrednosti $(\mathbf{x}, \mathbf{y})$
 neodvisno  vzorčene iz nabora $\mathcal{D}$ po porazdelitvi $P$. Po koncu učenja
 imamo torej na voljo funkcijo $\mathbf{h}(\mathbf{x})$, ki nam za nek vhodni nabor vrednosti
 $\mathbf{\hat{x}}$ poda napoved $\mathbf{\hat{y}}=\mathbf{h}(\mathbf{\hat{x}})$, ki ustrezno kategorizira
 ta nabor vrednosti. 
 
 Funkcije $\mathbf{h}$ so v praksi sestavljene iz (množice) preprostih funkcij z (nekaj) prostimi
 parametri, kar na koncu seveda pomeni velik skupni nabor neznanih parametrov in zahteven
 postopek minimizacije funkcije izgube. 
 
 Osnovni gradnik odločitvenih dreves je tako kar stopničasta funkcija $H(x_i-t_i)={0,1}$, ki je enaka
 ena za $x_i > t_i$ in nič drugače in kjer je $x_i$ ena izmed karakteristik in $t_i$ neznani parameter.
Iz skupine takšnih funkcij, ki predstavljajo binarne odločitve lahko skonstruiramo končno uteženo 
funkcijo $\mathbf{h}(\mathbf{x})=\sum\limits_{i=1}^{J} \mathbf{a}_{i}\, H(x_i-t_i),$ kjer so $\mathbf{a}_i$ vektorji neznanih uteži. 
Tako $t_i$ kot $\mathbf{b}_i$, lahko določimo v procesu učenja. Nadgradnjo predstavljajo nato \emph{pospešena} odločitvena
drevesa (BDT), kjer nadomestimo napoved enega drevesa z uteženo množico le-teh, tipično dobljeno
v ustreznih iterativnih postopkih (npr. AdaBoost, Gradient Boost ipd.).

Pri nevronskih mrežah je osnovni gradnik t.i. \emph{perceptron}, ki ga opisuje preprosta funkcija
$h_{w,b}(\mathbf{X})=\operatorname{\theta}\left(\mathbf{w}^{T} \cdot \mathbf{X} + b\right),$
kjer je $\mathbf{X}$ nabor vhodnih vrednosti, $\mathbf{w}$ vektor vrednosti uteži, s katerimi
tvorimo uteženo vsoto ter $b$ dodatni konstatni premik (bias). Funkcija $\theta$ je preprosta
gladka funkcija (npr. $\arctan$), ki lahko vpelje nelinearnost v odzivu perceptrona. Nevronska
mreža je nato sestavljena iz (poljubne) topologije takšnih perceptronov, ki na začetku
sprejme karakteristiko dogodka $\mathbf{x}$ v končni fazi rezultirajo v napovedi $\mathbf{\hat{y}}$, ki 
mora seveda biti čim bližje ciljni vrednosti $\mathbf{y}$. Z uporabo ustrezne funkcije 
izgube (npr MSE: $\mathcal{L}(h)=\mathbb{E}\, ||\mathbf{y}-\mathbf{\hat{y}}||^2$, se problem znova prevede na
minimizacijo, kjer iščemo optimalne vrednosti (velikega) nabora uteži $\mathbf{w}_i$ ter
$b_i$ za vse perceptrone v mreži. Globoke nevronske mreže (DNN) niso nič drugega, kot 
velike nevronske mreže ali skupine le-teh. 

Že namizni računalniki so
dovolj močni za  osnovne računske naloge, obstajajo pa tudi že zelo uporabniku prijazni vmesniki v jeziku Python, na primer:

  - Scikit-Learn (scikit-learn.org): odprtokodni paket za strojno učenje,
  - TensorFlow (tensorflow.org): odprtokodni Google-ov sistem za ML, s poudarkom na globokih nevronskih mrežah 
  ( Deep Neural Networks, DNN) z uporabo vmesnika Keras. Prilagojen za delo na GPU in TPU. 
  - Catboost: (Catboost.ai) : odprtokodna knjižnica za uporabo pospešenih odločitvenih dreves (Boosted Decision Trees, BDT). Prilagojena za delo na GPU.

Za potrebe naloge lahko uporabimo tudi spletni vmesnik Google Collab (colab.research.google.com), ki dopušča omejen dostop 
do večjih računskih zmogljivosti.


## Naloga

Na spletni učilnici je na voljo material (koda, vzorci) za 
ločevanje dogodkov Higgsovega bozona od ostalih procesov ozadja. V naboru simuliranih 
dogodkov je 18 karakteristik (zveznih kinematičnih lastnosti), katerih vsaka 
posamezno zelo slabo loči 'signal' od ozadja, z uporabo BDT ali (D)NN, pa lahko tu dosežemo
zelo dober uspeh.  Na predavanjih smo si ogledali glavne aspekte pomembne pri implementaciji ML, 
kot so  uporaba ustreznih spremenljivk (GIGO), učenje in prekomerno učenje (training/overtraining), 
vrednotenje uspeha metode kot razmerje med učinkovitostjo (efficiency) in čistostjo (precision) 
vzorca (Receiver Operating Characteristic, ROC). Določi uspešnost obeh metod (in nariši ROC) 
za nekaj tipičih konfiguracij BDT in DNN, pri čemer:

  - Študiraj vpliv uporabljenih vhodnih spremenljivk - kaj, če vzamemo le nekatere?
  - Študiraj BDT in NN in vrednoti uspešnost različnih nastavitev,  če spreminjaš nekaj konfiguracijskih parametrov
  (npr. število perceptronov in plasti nevronskih mrež pri DNN in število dreves pri BDT). 

**Dotatna naloga:**

Implementiraj distribucije iz ‘playground’ zgleda v BDT (lahko tudi RandomForests) in DNN,
te distribucije so na voljo v vseh popularnih ML paketih (npr. Scikit...).

In [1]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 83s 0us/step


2024-08-23 23:36:08.928379: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-08-23 23:36:08.928419: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-08-23 23:36:08.928431: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-08-23 23:36:08.928444: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-23 23:36:08.928456: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/5


2024-08-23 23:36:15.145657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


782/782 ━━━━━━━━━━━━━━━━━━━━ 625s 752ms/step - accuracy: 0.0564 - loss: 5.1049
Epoch 2/5
  4/782 ━━━━━━━━━━━━━━━━━━━━ 14:10 1s/step - accuracy: 0.1276 - loss: 4.0239   

KeyboardInterrupt: 

## Zaključek

---
Luka Skeledžija, [Github source 🔗](https://github.com/lukaske/mafijski-praktikum/tree/main/MFPR/12_mlearn), 2024